In [2]:
import os
import pandas as pd


In [3]:
%pwd

'/workspaces/chicken-disease-classification/research'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfiguratonManger:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):
        print("Configuration Manager Initiated")
        print(f"Config File Path: {config_filepath}, with dtype {type(config_filepath)}")
        print(f"Params File Path: {params_filepath} with dtype {type(params_filepath)}")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file with the folowing headers: \n{headers}")
        
        else:
            logger.info(f"File already exists at {self.config.local_data_file} of size {get_size(self.config.local_data_file)}")

        def extract_zip_file(self):

            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with ZipFile(self.config.local_data_file, 'r') as zip:
                zip.extractall(path=unzip_path)
                logger.info(f"Extracted file at {self.config.unzip_dir}")
                

In [17]:
try:
    config = ConfiguratonManger()
    data_ingestiom_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestiom_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e
    

Configuration Manager Initiated
Config File Path: config/config.yaml, with dtype <class 'pathlib.PosixPath'>
Params File Path: params.yaml with dtype <class 'pathlib.PosixPath'>
[ 2023-09-12 07:02:43,586: ERROR: 1807712840:  [Errno 2] No such file or directory: 'config/config.yaml']
Traceback (most recent call last):
  File "/tmp/ipykernel_2681/1807712840.py", line 2, in <module>
    config = ConfiguratonManger()
  File "/tmp/ipykernel_2681/140050448.py", line 10, in __init__
    self.config = read_yaml(config_filepath)
  File "/workspaces/chicken-disease-classification/env/lib/python3.10/site-packages/ensure/main.py", line 849, in __call__
    return_val = self.f(*args, **kwargs)
  File "/workspaces/chicken-disease-classification/src/cnnClassifier/utils/common.py", line 41, in read_yaml
    except Exception as ex:
  File "/workspaces/chicken-disease-classification/src/cnnClassifier/utils/common.py", line 33, in read_yaml
    with open(path_to_yaml, "w", encoding='utf-8') as yaml_file:

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'